### Configuração Inicial

In [14]:
import pathlib
import sys
requirements_filename = 'requirements_ap2.txt'
if str(pathlib.Path().resolve().name) == "notebooks":
    root = str(pathlib.Path().resolve().parent.parent)+'/'
    sys.path.append(root)
    requirements_path = root + 'requirements/'+requirements_filename
else:
    ! git clone https://github.com/Fabio-Trindade/Eng-Aprendizado-Maquina.git
    root = str(pathlib.Path().resolve())
    src_path = root + '/Eng-Aprendizado-Maquina/'
    requirements_path = src_path + 'requirements/'+requirements_filename
    sys.path.append(root + '/Eng-Aprendizado-Maquina/')

! pip install -r $requirements_path


### Imports

In [15]:
from src.utils.util_read_file import UtilReadFile
from src.utils.util_path import UtilPath
from src.constants.KPaths import KPaths
from src.entities.mysql import MySQL
from src.utils.util_api import UtilAPI
import pandas as pd
import os

In [16]:

root_path = UtilPath.get_root_path()

## Leitura de arquivos JSON, XLS, CSV e Parquet
Para ler arquivos nos formatos json, xls, csv e parquet foram utilizadas as biliotecas pandas (_pd_), pyarrow e parquet (_pq_). As funções que realizam estas leituras são simples e podem ser vistas na classe abaixo:

```python
class UtilReadFile:
    @staticmethod
    def read_json(filename) -> pd.DataFrame:
        with open(filename,'r') as file:
         return pd.DataFrame(json.load(file))
        
    
    @staticmethod
    def read_csv_with_pandas(filename) -> pd.DataFrame:
       return pd.read_csv(filename)
    
    @staticmethod
    def read_excel_with_pandas(filename) -> pd.DataFrame:
       return pd.read_excel(filename)
    
    @staticmethod
    def read_parquet(filename)-> pd.DataFrame:
       return pq.read_table(filename).to_pandas()     
``` 

### Lendo os dados em _titanic\_train.csv_ 

In [17]:
df = UtilReadFile.read_csv_with_pandas(KPaths.path_titanic_csv)
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


### Lendo os dados em _iris.json_

In [18]:
json_data1 = UtilReadFile.read_json(KPaths.path_iris_json)
df1 = pd.DataFrame(json_data1)
df1

,sepalLength,sepalWidth,petalLength,petalWidth,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


### Lendo dados em _champion\_info.json_

In [19]:
json_data2 = UtilReadFile.read_json(KPaths.path_champion_info_json)
df2 = pd.DataFrame(json_data2)
df2

,type,version,data
1,champion,7.17.2,"{'title': 'the Dark Child', 'id': 1, 'key': 'A..."
2,champion,7.17.2,"{'title': 'the Berserker', 'id': 2, 'key': 'Ol..."
3,champion,7.17.2,"{'title': 'the Colossus', 'id': 3, 'key': 'Gal..."
4,champion,7.17.2,"{'title': 'the Card Master', 'id': 4, 'key': '..."
5,champion,7.17.2,"{'title': 'the Seneschal of Demacia', 'id': 5,..."
...,...,...,...
429,champion,7.17.2,"{'title': 'the Spear of Vengeance', 'id': 429,..."
432,champion,7.17.2,"{'title': 'the Wandering Caretaker', 'id': 432..."
497,champion,7.17.2,"{'title': 'The Charmer', 'id': 497, 'key': 'Ra..."
498,champion,7.17.2,"{'title': 'the Rebel', 'id': 498, 'key': 'Xaya..."


### Processando os dados
O código abaixo transforma os dados dentro da coluna 'data' em novos dados, onde as chaves se tornaram colunas de um dataframe e os valores preencheram a sua respectiva coluna.

In [20]:
partial_df = df2['data']
dictionary = {  }

for column in list(dict(partial_df[0]).keys()):
    dictionary[column] = [ ] 

for i,str_dic in enumerate(partial_df.tolist()):
    dic = dict(str_dic)
    for column in list(dictionary.keys()):
            dictionary[column].append(dic[column])

df1_aux = pd.DataFrame(dictionary).reset_index()
df2_aux = df2.drop('data',axis=1).reset_index()
pd.concat([df1_aux,df2_aux],axis=1).drop('index',axis=1)


/tmp/ipykernel_703395/1453154378.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for column in list(dict(partial_df[0]).keys()):


,title,id,key,name,type,version
0,the Dark Child,1,Annie,Annie,champion,7.17.2
1,the Berserker,2,Olaf,Olaf,champion,7.17.2
2,the Colossus,3,Galio,Galio,champion,7.17.2
3,the Card Master,4,TwistedFate,Twisted Fate,champion,7.17.2
4,the Seneschal of Demacia,5,XinZhao,Xin Zhao,champion,7.17.2
...,...,...,...,...,...,...
133,the Spear of Vengeance,429,Kalista,Kalista,champion,7.17.2
134,the Wandering Caretaker,432,Bard,Bard,champion,7.17.2
135,The Charmer,497,Rakan,Rakan,champion,7.17.2
136,the Rebel,498,Xayah,Xayah,champion,7.17.2


### Lendo _PIB\_municipios.xls_

In [21]:
df3 = UtilReadFile.read_excel_with_pandas(KPaths.path_pib_mun_xls)
df3

,Ano,Código da Grande Região,Nome da Grande Região,Código da Unidade da Federação,Sigla da Unidade da Federação,Nome da Unidade da Federação,Código do Município,Nome do Município,Região Metropolitana,Código da Mesorregião,...,"Valor adicionado bruto dos Serviços, a preços correntes - exclusive Administração, defesa, educação e saúde públicas e seguridade social\n(R$ 1.000)","Valor adicionado bruto da Administração, defesa, educação e saúde públicas e seguridade social\n(R$ 1.000)","Valor adicionado bruto total, a preços correntes\n(R$ 1.000)","Impostos, líquidos de subsídios, sobre produtos, a preços correntes\n(R$ 1.000)","Produto Interno Bruto, a preços correntes\n(R$ 1.000)",População\n(Nº de habitantes),"Produto Interno Bruto per capita\n(R$ 1,00)",Atividade com maior valor adicionado bruto,Atividade com segundo maior valor adicionado bruto,Atividade com terceiro maior valor adicionado bruto
0,2010,1,Norte,11,RO,Rondônia,1100015,Alta Floresta D'Oeste,NaN,1102,...,62496.185,93244.656,241119.767,20957.111,262076.878,24422,10731.18,"Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços
1,2010,1,Norte,11,RO,Rondônia,1100023,Ariquemes,NaN,1102,...,494946.267,343867.731,1199664.227,165029.553,1364693.780,90354,15103.86,"Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...
2,2010,1,Norte,11,RO,Rondônia,1100031,Cabixi,NaN,1102,...,12677.210,25170.235,65400.772,4210.342,69611.114,6309,11033.62,"Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços
3,2010,1,Norte,11,RO,Rondônia,1100049,Cacoal,NaN,1102,...,465447.325,298454.309,1041212.374,145281.717,1186494.091,78601,15095.15,"Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...
4,2010,1,Norte,11,RO,Rondônia,1100056,Cerejeiras,NaN,1102,...,80724.991,63018.270,192454.160,29567.029,222021.189,17030,13037.06,"Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24932,2014,3,Sudeste,31,MG,Minas Gerais,3137007,Ladainha,NaN,3104,...,21515.923,54926.074,90887.431,1551.868,92439.299,17881,5169.69,"Administração, defesa, educação e saúde públic...",Demais serviços,"Agricultura, inclusive apoio à agricultura e a..."
24933,2014,3,Sudeste,31,MG,Minas Gerais,3137106,Lagamar,NaN,3101,...,38508.686,27315.218,127867.862,6688.350,134556.212,7800,17250.80,Demais serviços,"Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária"
24934,2014,3,Sudeste,31,MG,Minas Gerais,3137205,Lagoa da Prata,NaN,3106,...,670364.622,154077.408,1113219.689,181944.559,1295164.248,49654,26083.78,Demais serviços,Comércio e reparação de veículos automotores e...,Indústrias de transformação
24935,2014,3,Sudeste,31,MG,Minas Gerais,3137304,Lagoa dos Patos,NaN,3102,...,8102.773,16733.055,35844.904,1000.209,36845.113,4285,8598.63,"Administração, defesa, educação e saúde públic...",Demais serviços,"Pecuária, inclusive apoio à pecuária"


### Lendo _user\_tracking.parquet_

In [22]:
df4 = UtilReadFile.read_parquet(KPaths.path_user_track_parquet)
df4

,installation_number,external_identifier,handset,os,os_version,host_app_package,host_app_version,lib_version,config_version,advertising_id,...,iccid,ssid,type,bssid,ip,mac_address,fingerprint,is_rooted,is_mock_location,is_from_emulator
0,7206043,,samsung:SM-J700M,ANDROID,6.0.1,com.fiat.uconnectmobile,5.4.3,2.2.18,[new]fca_1.1,35487d8e-0b56-4d51-9b9e-e22179e188d0,...,8952050001802717893,,0,None,IPv4: 102.22.219.13 | IPv6: FE80::D4AE:5FF:FEB...,D4:AE:05:B3:4C:2F,samsung/j7elteub/j7elte:6.0.1/MMB29K/J700MUBU2...,NaN,NaN,NaN
1,6219664,367738313,LENOVO:Lenovo K33b36,None,6.0.1,com.pontomobi.smiles,None,2.2.17.2,None,None,...,None,"""HomeNet""",0,None,None,None,None,NaN,NaN,NaN
2,6485586,,samsung:SM-G928G,None,7.0,com.pontomobi.smiles,None,2.2.17.2,None,None,...,None,"""Costura_Casa da Lua""",0,None,None,None,None,NaN,NaN,NaN
3,7050958,108659666,samsung:SM-G930F,None,7.0,com.pontomobi.smiles,None,2.2.17,None,None,...,None,"""716""",0,None,None,None,None,NaN,NaN,NaN
4,7131516,108027625,asus:ASUS_X013D,None,5.1.1,com.pontomobi.smiles,None,2.2.17.2,None,None,...,None,<unknown ssid>,0,None,None,None,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28291,7210849,,motorola:Moto G (5) Plus,ANDROID,8.1.0,com.tenda.app,2.36,2.2.18,[new]Tenda_1.2,7dad27f4-3814-4ae5-a1a9-e9d056f23a0e,...,UNKNOWN,"""ASUScarol01""",1,20:cf:30:97:12:a0,IPv4: 192.168.2.4 | IPv6: FE80::50CC:6AFF:FE28...,F0:D7:AA:B8:DF:88,motorola/potter_nt/potter_nt:8.1.0/OPS28.85-17...,NaN,NaN,NaN
28292,6660693,441385055,motorola:Moto Z2 Play,None,8.0.0,com.pontomobi.smiles,None,2.2.17.2,None,None,...,None,<unknown ssid>,2,None,None,None,None,NaN,NaN,NaN
28293,6848495,506809026,samsung:SM-J700M,None,6.0.1,com.pontomobi.smiles,None,2.2.17.2,None,None,...,None,"""Paulinha""",2,None,None,None,None,NaN,NaN,NaN
28294,7110078,,motorola:Moto G (5S),None,7.1.1,com.pontomobi.smiles,None,2.2.17.2,None,None,...,None,"""VIVOFIBRA-3ED0-5G""",0,None,None,None,None,NaN,NaN,NaN


### Lendo _cliente\_cidade.sql_
A classe MySQL estabelece uma conexão(_get_\_connection()_) com o MySQL server e executar as _querys_(_execute\_query()_):
```python
class MySQL:
    def __init__(self,host,user,password,database):
        self.password = password
        self.host = host
        self.user = user
        self.database =  database
        
    def get_connection(self):
        try:
            connection = mysql.connector.connect(
            host=self.host,
            user=self.user,
            password=self.password,
            database=self.database )
            if connection.is_connected():
                return connection
        except mysql.connector.Error as err:
            print("Erro ao conectar ao banco de dados:", err)
        

    def execute_query(self, query:str):
        connection = self.get_connection()
        cursor = connection.cursor()
        cursor.execute(query)
        data = cursor.fetchall()
        cursor.close()
        connection.close()
        return data
    
```

Para que o arquivo _cliente\_cidade.sql_ seja lido com sucesso, é necessário:
* Criar um arquivo .env dentro do repositório _Eng-Aprendizado-De-Maquina_ e inserir os valores das variavés necessárias (PASSWORD, HOST, DATABASE e USER).
  * Ou simplesmente substitua as variáveis de ambiente abaixo pelos seus devidos valores. Por ex., substituir _os.getenv("HOST")_ por _"seu_host"_. 
* Criar o banco de dados com o arquivo _cliente\_cidade.sql_ (em _Eng-Aprendizado-Maquina/datasets/_) e que a porta do servidor seja _3306_.

In [23]:
password = os.getenv("PASSWORD")
host = os.getenv("HOST")
database = os.getenv("DATABASE")
user = os.getenv("USER")

mysql_connector = MySQL(host=host,user=user,password=password,database=database)

In [24]:
try:
  results = mysql_connector.execute_query("SELECT * FROM Cliente;")
  for id,nome,codIBGE in results:
    print(f"ID: {id} - Nome: {nome} - Código IBGE: {codIBGE}")
except Exception as e:
  print("Certifique de seguir as intruções da célula markdown acima")

ID: 7 - Nome: Prefeitura Municipal de Aimorés - Código IBGE: 3101102
ID: 16 - Nome: Prefeitura Municipal de Argirita - Código IBGE: 3104403
ID: 20 - Nome: Prefeitura Municipal de Bandeira - Código IBGE: 3105202
ID: 21 - Nome: Prefeitura Municipal de Barão de Cocais - Código IBGE: 3105400
ID: 22 - Nome: Prefeitura Municipal de Bela Vista de Minas - Código IBGE: 3106002
ID: 23 - Nome: Prefeitura Municipal de Belo Oriente - Código IBGE: 3106309
ID: 26 - Nome: Prefeitura Municipal de Bom Jesus do Galho - Código IBGE: 3107802
ID: 33 - Nome: Câmara Municipal de Bandeira - Código IBGE: None
ID: 35 - Nome: Câmara Municipal de Braúnas - Código IBGE: None
ID: 40 - Nome: Câmara Municipal de Coronel Fabriciano - Código IBGE: None
ID: 43 - Nome: Câmara de Governador Valadares - Código IBGE: None
ID: 46 - Nome: Câmara Municipal de Itabira - Código IBGE: None
ID: 47 - Nome: Câmara Municipal de Lagoa Santa - Código IBGE: None
ID: 50 - Nome: Custom - Câmara Municipal de Nova Serrana - Código IBGE: None

### Lendo dados da API do IBGE

A classe UtilAPI abaixo basicamente executa uma requisição através do _endpoint_ de uma API.
```python
class UtilAPI:
    @staticmethod
    def get(url):
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            raise ValueError('Erro: ', response.status_code)
```

Os códigos abaixo extraem e mostram os nomes de todos os municípios de Minas Gerais através da API disponibilizada pelo IBGE.

In [25]:
results = UtilAPI.get("https://servicodados.ibge.gov.br/api/v1/localidades/estados/MG/municipios")
print('Info dos dados obtidos')
results[0]

Info dos dados obtidos


{'id': 3100104,
 'nome': 'Abadia dos Dourados',
 'microrregiao': {'id': 31019,
  'nome': 'Patrocínio',
  'mesorregiao': {'id': 3105,
   'nome': 'Triângulo Mineiro/Alto Paranaíba',
   'UF': {'id': 31,
    'sigla': 'MG',
    'nome': 'Minas Gerais',
    'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}},
 'regiao-imediata': {'id': 310061,
  'nome': 'Monte Carmelo',
  'regiao-intermediaria': {'id': 3111,
   'nome': 'Uberlândia',
   'UF': {'id': 31,
    'sigla': 'MG',
    'nome': 'Minas Gerais',
    'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}}}

In [26]:
for municipio in results:
    print("Nome do município: ",municipio['nome'])

Nome do município:  Abadia dos Dourados
Nome do município:  Abaeté
Nome do município:  Abre Campo
Nome do município:  Acaiaca
Nome do município:  Açucena
Nome do município:  Água Boa
Nome do município:  Água Comprida
Nome do município:  Aguanil
Nome do município:  Águas Formosas
Nome do município:  Águas Vermelhas
Nome do município:  Aimorés
Nome do município:  Aiuruoca
Nome do município:  Alagoa
Nome do município:  Albertina
Nome do município:  Além Paraíba
Nome do município:  Alfenas
Nome do município:  Alfredo Vasconcelos
Nome do município:  Almenara
Nome do município:  Alpercata
Nome do município:  Alpinópolis
Nome do município:  Alterosa
Nome do município:  Alto Caparaó
Nome do município:  Alto Rio Doce
Nome do município:  Alvarenga
Nome do município:  Alvinópolis
Nome do município:  Alvorada de Minas
Nome do município:  Amparo do Serra
Nome do município:  Andradas
Nome do município:  Cachoeira de Pajeú
Nome do município:  Andrelândia
Nome do município:  Angelândia
Nome do municíp